# Import Modules


In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import warnings
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img, array_to_img
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras import layers, Sequential

warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'numpy'

## Load the files

In [2]:
BASE_DIR = 'C:/Users/ASUS/OneDrive/Desktop/Python_Course/data'

In [3]:
# load all the complete  image paths to the list
image_paths =[]
for image_name in os.listdir(BASE_DIR):
    image_path = os.path.join(BASE_DIR,image_name)
    image_paths.append(image_path)

In [4]:
image_paths[:5]

['C:/Users/ASUS/OneDrive/Desktop/Python_Course/data\\1.png',
 'C:/Users/ASUS/OneDrive/Desktop/Python_Course/data\\10.png',
 'C:/Users/ASUS/OneDrive/Desktop/Python_Course/data\\100.png',
 'C:/Users/ASUS/OneDrive/Desktop/Python_Course/data\\1000.png',
 'C:/Users/ASUS/OneDrive/Desktop/Python_Course/data\\10000.png']

In [5]:
len(image_paths)

21552

## Visualize the Images Dataset

In [6]:
# to display grid of images (8x8)
plt.figure(figsize=(20,20))
temp_images = image_paths[:64]
index = 1

for image_path in temp_images:
    plt.subplot(8,8,index)
    # load the image
    img = load_img(image_path)
    # convert to numpy array
    img = np.array(img)
    # show the image
    plt.imshow(img)
    plt.axis('off')
    # increament the index for next image
    index += 1

NameError: name 'plt' is not defined

## Preprocess the Images

In [7]:
# load the image and convert to numpy array
train_images = [np.array(load_img(path)) for path in tqdm (image_paths)]
train_images = np.array(train_images)

NameError: name 'tqdm' is not defined

In [8]:
train_images[0].shape

NameError: name 'train_images' is not defined

In [9]:
# reshape the array
train_images = train_images.reshape(train_images.shape[0],64,64,3).astype('float32')

NameError: name 'train_images' is not defined

In [10]:
# normalize the images
train_images = (train_images - 127.5)/127.5

NameError: name 'train_images' is not defined

In [11]:
train_images[0]

NameError: name 'train_images' is not defined

## Create  Generator And Discriminator Models


In [12]:
# latent dimension for random noise
LATENT_DIM = 100 
# Weight initializer
WEIGHT_INIT = keras.initializers.RandomNormal(mean = 0.0,stddev = 0.02)
# no.  of channel of the Image
CHANNELS = 3 # for grey scale, keep it as 1

NameError: name 'keras' is not defined

## Generator Model

Generator Model will create new Images similar to Training  data from  random noise

In [13]:
model = Sequential(name='generator')

# 1D random noise
model.add(layers.Dense(8 * 8 * 512, input_dim=LATENT_DIM))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())

# Convert 1D to 3D
model.add(layers.Reshape((8, 8, 512)))

# Upsample to 16x16
model.add(layers.Conv2DTranspose(256, (4, 4), strides=(2, 2), padding='same', kernel_initializer=WEIGHT_INIT))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())

# Upsample to 32x32
model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', kernel_initializer=WEIGHT_INIT))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())

# Upsample to 64x64
model.add(layers.Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same', kernel_initializer=WEIGHT_INIT))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())

# Final output layer
model.add(layers.Conv2D(CHANNELS, (4, 4), padding='same', activation='tanh'))


generator = model
generator.summary()

NameError: name 'Sequential' is not defined

## Discriminator Model

Discriminator Model will  classify the image from the generator to check wheather it real(or)fake images.

In [14]:
model = Sequential(name ='discriminator')
input_shape = (64,64,3)
alpha = 0.2

# create conv layers 
model.add(layers.Conv2D(64,(4,4),strides = (2,2),padding ='same', input_shape = input_shape))
model.add(layers.BatchNormalization())
model.add(layers.LeakyReLU(alpha = alpha))

model.add(layers.Conv2D(128,(4,4),strides = (2,2),padding ='same', input_shape = input_shape))
model.add(layers.BatchNormalization())
model.add(layers.LeakyReLU(alpha = alpha))

model.add(layers.Conv2D(128,(4,4),strides = (2,2),padding ='same', input_shape = input_shape))
model.add(layers.BatchNormalization())
model.add(layers.LeakyReLU(alpha = alpha))

model.add(layers.Flatten())
model.add(layers.Dropout(0.4))

# output class
model.add(layers.Dense(1,activation = 'sigmoid'))


discriminator = model
discriminator.summary()

NameError: name 'Sequential' is not defined

## Create DCGAN

In [15]:
import tensorflow as tf
from tensorflow import keras

class DCGAN(keras.Model):
    def __init__(self, generator, discriminator, latent_dim):
        super().__init__()
        self.generator = generator
        self.discriminator = discriminator
        self.latent_dim = latent_dim
        self.g_loss_metric = keras.metrics.Mean(name='g_loss')
        self.d_loss_metric = keras.metrics.Mean(name='d_loss')
        
    @property
    def metrics(self):
        return [self.g_loss_metric, self.d_loss_metric]
    
    def compile(self, g_optimizer, d_optimizer, loss_fn):
        super(DCGAN, self).compile()
        self.g_optimizer = g_optimizer
        self.d_optimizer = d_optimizer
        self.loss_fn = loss_fn
        
    def train_step(self, real_images):
        # Get batch size from the data
        batch_size = tf.shape(real_images)[0]
        # Generate random noise
        random_noise = tf.random.normal(shape=(batch_size, self.latent_dim))
        
        # Train the discriminator with real (1) and fake (0) images
        with tf.GradientTape() as tape:
            # Compute loss on real images
            pred_real = self.discriminator(real_images, training=True)
            # Generate real image labels with slight noise
            real_labels = tf.ones((batch_size, 1)) + 0.05 * tf.random.uniform(tf.shape(pred_real))
            d_loss_real = self.loss_fn(real_labels, pred_real)
            
            # Compute loss on fake images
            fake_images = self.generator(random_noise)
            pred_fake = self.discriminator(fake_images, training=True)
            fake_labels = tf.zeros((batch_size, 1))
            d_loss_fake = self.loss_fn(fake_labels, pred_fake)
            
            # Total discriminator loss
            d_loss = (d_loss_real + d_loss_fake) / 2
            
        # Compute discriminator gradients
        gradients = tape.gradient(d_loss, self.discriminator.trainable_variables)
        # Update the gradients
        self.d_optimizer.apply_gradients(zip(gradients, self.discriminator.trainable_variables))
        
        # Train the generator model
        labels = tf.ones((batch_size, 1))
        
        # Generator wants discriminator to think fake images are real
        with tf.GradientTape() as tape:
            # Generate fake images from generator
            fake_images = self.generator(random_noise, training=True)
            # Classify images as real or fake
            pred_fake = self.discriminator(fake_images, training=True)
            # Compute the loss
            g_loss = self.loss_fn(labels, pred_fake)
            
        # Compute generator gradients
        gradients = tape.gradient(g_loss, self.generator.trainable_variables)
        # Update the gradients
        self.g_optimizer.apply_gradients(zip(gradients, self.generator.trainable_variables))
        
        # Update states for both models
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        
        return {'d_loss': self.d_loss_metric.result(), 'g_loss': self.g_loss_metric.result()}

ModuleNotFoundError: No module named 'tensorflow'

In [16]:
class DCGANMonitor(keras.callbacks.Callback):
    def __init__(self, num_imgs=25, latent_dim=100):
        self.num_imgs = num_imgs
        self.latent_dim = latent_dim
        # Create random noise for generating images
        self.noise = tf.random.normal([25, latent_dim])

    def on_epoch_end(self, epoch, logs=None):
        # Generate images
        g_img = self.model.generator(self.noise)
        # Denormalize the images
        g_img = (g_img * 127.5) + 127.5
        g_img = g_img.numpy()

        fig = plt.figure(figsize=(8,8))
        for i in range(self.num_imgs):
            plt.subplot(5, 5, i + 1)  # 6x6 grid for 36 images
            img = array_to_img(g_img[i])
            plt.imshow(img)
            plt.axis('off')

        # Save and show the plot
        plt.savefig(f'epoch_{epoch:03d}.png')
        plt.show()
        plt.close(fig)

    def on_train_end(self, logs=None):
        # Save the generator model after training
        self.model.generator.save('generator.h5')

NameError: name 'keras' is not defined

In [17]:
dcgan = DCGAN(generator,discriminator,latent_dim = LATENT_DIM)

NameError: name 'DCGAN' is not defined

In [18]:
D_LR = 0.0001
G_LR = 0.0003

dcgan.compile(g_optimizer = Adam(learning_rate = G_LR, beta_1 = 0.5), d_optimizer = Adam(learning_rate = D_LR, beta_1 = 0.5),loss_fn = BinaryCrossentropy())

NameError: name 'dcgan' is not defined

In [19]:
N_EPOCHS = 50
dcgan.fit(train_images,epochs = N_EPOCHS, callbacks = [DCGANMonitor()])

NameError: name 'dcgan' is not defined

## Generate New Anime Image

In [20]:
noise = tf.random.normal([1,100])
fig = plt.figure(figsize=(2,2))
# Generate images
g_img = dcgan.generator(noise)
# Denormalize the images
g_img = (g_img * 127.5) + 127.5
g_img = g_img.numpy()

img = array_to_img(g_img[0])
plt.imshow(img)
plt.axis('off')

        # Save and show the plot
plt.savefig(f'epoch_{epoch:03d}.png')
plt.show()
plt.close(fig)

def on_train_end(self, logs=None):
    # Save the generator model after training
    self.model.generator.save('generator.h5')

NameError: name 'tf' is not defined